# Coli Toolkit (CTK) Flow Cytometry Analysis and Model Calibration

## User Setup

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import FlowCal as fcs
import os
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity
import shutil
import warnings

### Data Source Setup
Here you can link the directories containing the respective data.

In [ ]:
data_dir = "data/"
for rep_dir in os.listdir(data_dir):
    rep_dir_path = os.path.join(data_dir, rep_dir)
    if not os.path.isdir(rep_dir_path):
        continue    
    if not rep_dir.startswith("replicate"):
        continue
    
    for const_dir in os.listdir(rep_dir_path):
        const_dir_path = os.path.join(rep_dir_path, const_dir)
        if not os.path.isdir(const_dir_path):
            continue   

        target_dir = os.path.join(data_dir, const_dir, rep_dir)
        os.makedirs(target_dir, exist_ok=True)
        for file in os.listdir(const_dir_path):
            file_path = os.path.join(const_dir_path, file)
            if not os.path.isfile(file_path):
                continue

            target_file_path = os.path.join(target_dir, file)
            shutil.copyfile(file_path, target_file_path)
            print(f"Copied {file_path} to {target_file_path}")
            
    

In [ ]:
data_dir_constitutive = "data/constitutive/"
data_dir_inputs = "data/inputs/"
data_dir_gates = "data/gates/"
data_dir_reference = "data/reference/"
data_dir_basal = "data/basal/"
data_dir_inputs_cross_reactivity = "data/inputs_cross_reactivity/"


## Figure Output Setup

In [ ]:
figure_dir = "figures/"
figure_extension = ".pdf"

os.makedirs(figure_dir, exist_ok=True)

### General Setup

In [ ]:
reference_promoter = "J23101"                                           # The constitutive reference promoter which serves as baseline for the RPU conversion
alternative_reference_promoter_name = {"J23101": "pJCM434"}             # Currently, the reference promoter is part of the dataset in two different names
inducer_units = {"ind1": "pgul", "ind2": "uM", "ind3": "uM"}
inducer_molecule_type = {"pJCM435": "IPTG", "pJCM448": "Ara", "pJCM449": "aTc"}
controlling_input_sensor = "pJCM435"                                    # The input sensor used for the characterization of the gate plasmids
facs_channel = "FL1-A"                                                  # The fluorescence FACS channel we are interested in
p_gating = 0.95                                                         # The probability mass to preserve after gating
gating_channels = ["FSC-A", "FSC-H"]                                    # The two channels to apply gating on
kde_bandwith = 0.05                                                     # Bandwith of gaussian kernel density estimate

## Model Calibration Setup

In [ ]:
n_walkers = 10
n_chains = 10
n_samples = 10000

### Figures Setup

In [ ]:
lim = (10**(-3), 10**2)
histogram_bins = np.logspace(np.log10(lim[0]), np.log10(lim[1]), 200)                
hist_as_density = False                                                # Whether to turn the histogram into a valid density (sum over width x height = 1) or not
plot_as_density = False                                                 # Whether to use kernel density estimate instead of histograms for representing distributions

input_sensor_order = ["Ptac", "PBAD", "Ptet"]
inducer_order = ["No", "IPTG", "Ara", "aTc"]
present_top_k = 5                                                      # Number of top gates to present according to dynamic range

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

font = {'family': 'serif',
        'serif': ['Helvetica'],
        'size': 8}

savefig = {'bbox': 'tight',
           'pad_inches': 0.01,
           'dpi': 1200,
           'transparent': True}

matplotlib.rc('font', **font)
matplotlib.rc('savefig', **savefig)
mm_to_inch = lambda val: np.array(val) * 0.0393701

In [ ]:
COLORS = [(0.30, 0.56, 1.00), (0.35, 0.24, 1.00), (1.00, 0.00, 0.42), (1.00, 0.40, 0.10), (1.00, 0.65, 0.19)]
COLORS_DARK = [(0.00, 0.24, 0.90), (0.28, 0.00, 0.84), (0.80, 0.00, 0.27), (0.90, 0.24, 0.00), (1.00, 0.50, 0.00)]
COLORS_MAIN = [(0.30, 0.55, 1.00), (1.00, 0.40, 0.10)]
COLORS_CMAP_ORANGE = ["#FFFFFF", "#FF5500", "#B3003C"]
COLORS_CMAP_BLUE = ["#FFFFFF", "#69A3FF", "#4400D6"]

COLORMAP_ORANGE = LinearSegmentedColormap.from_list("my_cmap", COLORS_CMAP_ORANGE)
COLORMAP_BLUE = LinearSegmentedColormap.from_list("my_cmap", COLORS_CMAP_BLUE)

COLOR_GRAY = "#808080"
COLOR_REPLICATES = COLORS_DARK[2:] # Use only redish colors for replicates

## Definition of Utility Functions

In [ ]:
def list_facs_files(directory):
    replicate_directories = os.listdir(directory)                    # Gets all the elements in data_dir and stores them as list in files 

    relevant_files = []                             # Target list to store the relevant files in
    for rep_dir in replicate_directories:
        rep_dir_path = os.path.join(directory, rep_dir)
        if not os.path.isdir(rep_dir_path):
            continue
            
        files = os.listdir(rep_dir_path)
        # We here filter for .fcs files and store them in relevant_files        
        for file in files:
            if os.path.splitext(file)[-1] != ".fcs":    # Exclude not .fcs files            
                continue
            relevant_files.append(os.path.join(rep_dir_path, file))                 # Adds file to the list of relevant files
            # break
    return relevant_files

def merge_replicates(replicates_data):
    # Compare replicates via median values and discard whole dataset in case the replicates deviation is too large.    
    merged_data = None
    num_replicates = len(replicates_data)

    if num_replicates < 1:
        return merged_data
        
    num_levels = len(replicates_data[0])

    merged_data = [None] * num_levels
    merged_info = [None] * num_levels
    for iL in range(num_levels):
        dist_mat = np.zeros(shape=(num_replicates, num_replicates))
        print("Merge", iL, list(map(np.median, [replicate[iL] for replicate in replicates_data])))
        replicate_lengths = np.array([len(replicate[iL]) for replicate in replicates_data])
        replicates_to_consider = replicate_lengths >= 1000     # Require at least 1000 samples
        
        for iR1 in range(num_replicates):            
            rep_val_1 = np.median(replicates_data[iR1][iL])            
            for iR2 in range(num_replicates):
                if iR2 < iR1:
                    continue
                    
                rep_val_2 = np.median(replicates_data[iR2][iL])
                dist_mat[iR1, iR2] = rep_val_1 / rep_val_2
                dist_mat[iR2, iR1] = dist_mat[iR1, iR2]
        
        relative_distances = np.abs(np.log2(dist_mat))
        vals = np.triu(relative_distances, k=1)
        threshold = 3
        mask = relative_distances <= threshold
        use_replicates = np.all(mask[replicates_to_consider][:, replicates_to_consider])      # Only consider replicates with sufficient read counts
        indices_to_consider = np.logical_and(replicates_to_consider, use_replicates)
        
            
        print("Merge", iL, indices_to_consider)
        cur_data = [replicates_data[iR][iL] for iR, flag in enumerate(indices_to_consider) if flag]
        if len(cur_data) > 0:
            merged_data[iL] = np.concatenate(cur_data)
        else:
            merged_data[iL] = np.array([])
        merged_info[iL] = indices_to_consider
    return merged_data, merged_info
                
    

def transform_data_dict(data_dict, construct_inducer_units={}):
    for construct_id in data_dict:
        print(construct_id)
        replicates = {}
        for replicate_id in data_dict[construct_id]:
            cur_data = data_dict[construct_id][replicate_id]
            replicate_inducer_unit = ""
            replicate_inducer_molecule = ""
            if construct_id in construct_inducer_units:
                replicate_inducer_unit = construct_inducer_units[construct_id]["unit"]
                replicate_inducer_molecule = construct_inducer_units[construct_id]["molecule"]
                
            replicate_inducer_concentrations = list(cur_data.keys())
            replicate_inducer_concentrations = sorted(replicate_inducer_concentrations)            
            
            fcs_datasets = [cur_data[conc] for conc in replicate_inducer_concentrations]
            replicate_data = [fcs_data[:, facs_channel] for fcs_data in fcs_datasets]
            replicates[replicate_id] = {"inducer unit": replicate_inducer_unit,
                                        "inducer molecule": replicate_inducer_molecule,
                                        "inducer concentrations": replicate_inducer_concentrations, 
                                        "data": replicate_data}

        replicate_inducer_units = [replicates[rep_id]["inducer unit"] for rep_id in replicates]
        replicate_inducer_molecules = [replicates[rep_id]["inducer molecule"] for rep_id in replicates]
        if len(set(replicate_inducer_units)) > 1:
            raise Exception("Replicates have different inducer units")
        if len(set(replicate_inducer_molecules)) > 1:
            raise Exception("Replicates have different inducer molecules")
            
        inducer_unit = replicate_inducer_units[0]
        inducer_molecule = replicate_inducer_molecules[0]

        replicate_inducer_concentrations = [replicates[rep_id]["inducer concentrations"] for rep_id in replicates]
        if not all([elem == replicate_inducer_concentrations[0] for elem in replicate_inducer_concentrations]):
            raise Exception("Not all replicates have the same inducer concentrations")
        
        inducer_concentrations = replicate_inducer_concentrations[0]

        replicates_data = [replicates[rep_id]["data"] for rep_id in replicates]
        merged_data, merged_info = merge_replicates(replicates_data)        
        data = merged_data
        
        data_dict[construct_id] = {"inducer unit": inducer_unit,
                                   "inducer molecule": inducer_molecule,
                                   "inducer concentrations": inducer_concentrations,
                                   "data unit": "AU",
                                   "data": data,
                                   "replicates merge info": merged_info, 
                                   "replicates": replicates}

def load_constitutive_data(file_paths):
    data_dict = {}             # Dictionary for storing the data with the inducer level as key
    for path in file_paths:                                              # Iterate over the facs files
        facs_data = fcs.io.FCSData(path)                                 # Use library to load facs data    
        facs_data = fcs.gate.density2d(facs_data,                        # Gate facs data
                                           channels=gating_channels,
                                           gate_fraction=p_gating)       
        # Extract and parse file name
        replicate_id = os.path.basename(os.path.split(path)[0])
        filename = os.path.basename(path)
        name, ext = os.path.splitext(filename)
        if not name in data_dict:
            data_dict[name] = {}
        data_dict[name][replicate_id] = {np.nan: facs_data}
        print(f"Loaded data for `{name}` ({replicate_id})")


    transform_data_dict(data_dict)
    return data_dict


def load_inducible_data(file_paths):
    
    data_dict = {}             # Dictionary for storing the data with the inducer level as key
    construct_inducer_units = {}
    
    for path in file_paths:                                              # Iterate over the facs files
        facs_data = fcs.io.FCSData(path)                                 # Use library to load facs data    
        facs_data = fcs.gate.density2d(facs_data,                        # Gate facs data
                                           channels=gating_channels,
                                           gate_fraction=p_gating)
        
        # Extract and parse file name
        replicate_id = os.path.basename(os.path.split(path)[0])
        filename = os.path.basename(path)
        name, ext = os.path.splitext(filename)
    
        # Extract and parse file name
        # Exemplary content of filename: pJCM435 0uM.fcs
        # Format is: "[construct_id] [inducer_level][inducer_unit].fcs"
        construct_id, inducer_description = name.split(" ")
        
        inducer_unit = None        
        for unit in inducer_units.values():      
            
            if unit in inducer_description:
                inducer_unit = unit
        if inducer_unit is None:
            raise Exception(f"Unknow inducer unit in {inducer_description}")
    
        inducer_concentration = float(inducer_description.replace(inducer_unit, ""))
        
        if not construct_id in data_dict:
            data_dict[construct_id] = {}
            construct_inducer_units[construct_id] = {"molecule": inducer_molecule_type[construct_id] if construct_id in inducer_molecule_type else "", 
                                                     "unit": inducer_unit}
            
        if not replicate_id in data_dict[construct_id]:
            data_dict[construct_id][replicate_id] = {}
        
        data_dict[construct_id][replicate_id][inducer_concentration] = facs_data
        
        print(f"Loaded data for construct `{construct_id}` and inducer concentration {inducer_concentration} {inducer_unit} ({replicate_id})")
    
    
    transform_data_dict(data_dict, construct_inducer_units)
    return data_dict


## Load and Gate Data

### Load Constitutive Data

In [ ]:
# We here infer the inducer concentration from the file names and load the facs data

file_paths = list_facs_files(data_dir_constitutive)
file_paths = sorted(file_paths)
# file_paths = [path for path in file_paths if "J23107" in path]   # Exemplary code for loading data of a single construt only

data_constitutive = load_constitutive_data(file_paths)
rpu_reference_data = data_constitutive[reference_promoter + "-No"]["data"]

### Load Input Data

In [ ]:
# We here infer the inducer concentration from the file names and load the facs data

file_paths = list_facs_files(data_dir_inputs)
file_paths = sorted(file_paths)

data_inputs = load_inducible_data(file_paths)

### Load Gate Data

In [ ]:
file_paths = list_facs_files(data_dir_gates)
file_paths = sorted(file_paths)
# file_paths = [path for path in file_paths if "pJCM212" in path] # Exemplary code for loading data of a single gate construct

data_gates = load_inducible_data(file_paths)

### Load Reference Data

In [ ]:
# We here infer the inducer concentration from the file names and load the facs data

file_paths = list_facs_files(data_dir_reference)
file_paths = sorted(file_paths)

data_reference = load_inducible_data(file_paths)
ref_name = alternative_reference_promoter_name[reference_promoter]

### Load Autofluorescence Data

In [ ]:
file_paths = list_facs_files(data_dir_basal)
file_paths = sorted(file_paths)

data_basal = load_inducible_data(file_paths)
autofluorescence_reference_data = np.concatenate((data_basal["DH10B"]["data"])) # Use data from all inducer concentrations as reference

### Load Input Cross Reactivitiy Data

In [ ]:
# We here infer the inducer concentration from the file names and load the facs data

file_paths = list_facs_files(data_dir_inputs_cross_reactivity)
file_paths = sorted(file_paths)

data_inputs_cross_reactivity = load_constitutive_data(file_paths)

## Normalize Data to RPU

Equation for the scaling factor $c$ used to rescale the fluorescence values into RPU.

\begin{align*}
c = \frac{\bar{FP} - \bar{FP}_0}{\bar{FP}} \, \frac{1}{\bar{FP}_{RPU} - \bar{FP}_0}
\end{align*}

In particular, $\bar{FP}$ is the median fluorescence of the expression values under consideration. $\bar{FP}_0$ is the median autofluorescence and $\bar{FP}_{RPU}$ is the median fluorescence of the reference plasmid.

In [ ]:
def transform_to_RPU(data_dict, rpu_reference_data, autofluorescence_reference_data):
    def rpu_transform(data, FP_0, FP_RPU):
        output_data = [None] * len(data)
        for iX in range(len(data)):
            FP = np.median(data[iX])
            
            c = (FP - FP_0)/FP * 1 / (FP_RPU - FP_0)
            
            output_data[iX]= data[iX] * c        
        return output_data
        
    FP_RPU = np.median(rpu_reference_data)
    FP_0 = np.median(autofluorescence_reference_data)
    # print(FP_RPU)
    for construct_id in data_dict:
        cur_entry = data_dict[construct_id]
  
            
        data = cur_entry["data"]        
        cur_entry["output"] = rpu_transform(data, FP_0, FP_RPU)
        if "replicates" in cur_entry:
            for replicate_id in cur_entry["replicates"]:
                data =  cur_entry["replicates"][replicate_id]["data"]        
                cur_entry["replicates"][replicate_id]["output"] = rpu_transform(data, FP_0, FP_RPU)

        

In [ ]:
transform_to_RPU(data_constitutive, rpu_reference_data, autofluorescence_reference_data)
transform_to_RPU(data_inputs, rpu_reference_data, autofluorescence_reference_data)
transform_to_RPU(data_gates, rpu_reference_data, autofluorescence_reference_data)
transform_to_RPU(data_reference, rpu_reference_data, autofluorescence_reference_data)
transform_to_RPU(data_basal, rpu_reference_data, autofluorescence_reference_data)
transform_to_RPU(data_inputs_cross_reactivity, rpu_reference_data, autofluorescence_reference_data)

## Insert Inducer RPU into Controlled Constructs
In particular, the median RPU of the input sensor used for characterizing is added to the controlled constructs data as second input reference. The original inducer concentration will be preserved.

In [ ]:
def insert_input_rpu(data_dict, input_data):
    median_input_RPU = [np.median(elem) for elem in input_data["output"]]
    for construct_id in data_dict:
        data_dict[construct_id]["input RPU"] = median_input_RPU  
        # Theoretically, one could assign the RPU histograms instead of the median value.
        # However, the combined data wouldn't represent actuall joint data as the data has been generated in two separate experiments.

In [ ]:
input_data = data_inputs[controlling_input_sensor]

insert_input_rpu(data_gates, input_data=input_data)

# Model Calibration
For the model calibration, we use the inhibitory Hill equation as model for the response curve and Parallel Tempering, a Markov chain Monte Carlo algorithm, for parameter sampling and optimization.

In [ ]:
from tqdm import tqdm

class OptimizationAlgorithm:

    def __init__(self):
        pass


    def run(self):
        raise Exception("Needs to be implemented by sub class")

class ParallelTempering(OptimizationAlgorithm):

    def __init__(self, log_likelihood, log_prior, n_dim, n_walkers=1, n_chains=10):
        self.log_likelihood = log_likelihood
        self.log_prior = log_prior
        self.n_dim = n_dim

        self.n_walkers = n_walkers
        self.n_chains = n_chains

        swap_mask = np.zeros(shape=(n_walkers, int(np.ceil(n_chains / 2) * 2)), dtype=bool)
        swap_mask[:, ::2] = 1
        self.swap_mask = swap_mask
        pass

    def run(self, initial_parameters, n_samples=10 ** 3, target_acceptance_ratio=None,
            adaptive_temperature=True):

        n_walkers = self.n_walkers
        n_chains = self.n_chains

        initial_parameters = np.array(initial_parameters)
        self.temperatures = np.power(2, np.arange(self.n_chains), dtype=float)

        if adaptive_temperature and n_chains <= 2:
            print(
                f"Disabling adaptive temperature for n_chains={n_chains}. Minimal number of chains for adaptive temperature is 3, but more chains are recommended.")
            adaptive_temperature = False

        if adaptive_temperature:
            self.temperatures[-1] = np.inf
            # Value choice follows Vousden et al. 2016
            v_factor = 10 ** 2
            v = int(np.ceil(v_factor / n_walkers))
            t0 = 10 * v  
            S = np.log(np.diff(self.temperatures, axis=-1))
            S = S[:-1]
            # Diffs of T_2 - T_1, ..., T_(N-1) - T_(N-2). The diff T_N - T_(N-1) is excluded by purpose following 1 < i < N for the S_i

        variance = 0.1
        self.variance = np.ones(shape=(self.n_walkers, self.n_chains, self.n_dim))
        self.variance = self.variance * np.expand_dims(np.expand_dims(np.arange(1, self.n_chains + 1), axis=0),
                                                       axis=-1)
        self.variance *= variance

        adaptive_proposal_distribution = target_acceptance_ratio is not None and target_acceptance_ratio > 0 and target_acceptance_ratio < 1.0

        adaptive_temperature_stop_iteration = int(n_samples / 2)

        parameters = np.zeros(shape=(n_samples, n_walkers, n_chains, *initial_parameters.shape))
        priors = np.zeros(shape=(n_samples, n_walkers, n_chains))
        likelihoods = np.zeros(shape=(n_samples, n_walkers, n_chains))
        step_accepts = np.zeros(shape=(n_samples, n_walkers, n_chains))
        swap_accepts = []

        params = np.array(initial_parameters)
        likelihood = self.log_likelihood(params)
        prior = self.log_prior(params)
        # max_iN = 0
        for iN in tqdm(range(n_samples)):
            self.beta = 1 / np.expand_dims(self.temperatures, axis=0)

            params, prior, likelihood, step_accept = self.step(params, prior, likelihood, index=iN)
            swap_round = iN % 10 == 9
            if swap_round:
                params, prior, likelihood, swap_accept = self.swap(params, prior, likelihood, index=iN)
                swap_accepts.append(swap_accept)

            parameters[iN] = params
            priors[iN] = prior
            likelihoods[iN] = likelihood
            step_accepts[iN] = step_accept

            ##################################
            # Adaptive Proposal Distribution #
            ##################################
            if adaptive_proposal_distribution and iN >= 100 and iN % 10 == 0:
                # Considers Windowed average of the last 100 steps
                acc_rate_deviation = step_accepts[max(iN - 100 + 1, 0):iN + 1] - target_acceptance_ratio
                scaling_params = np.exp((np.mean(acc_rate_deviation, axis=0)))
                self.variance = self.variance * np.expand_dims(scaling_params, axis=-1)

            ###############################
            # Adaptive Temperature Ladder #
            ###############################

            if adaptive_temperature and swap_round and iN > 20 and iN < adaptive_temperature_stop_iteration:
                kappa = 1 / v * t0 / (iN + t0)
                # Be aware that only every 10th iteration is a swap iteration
                rel_accepts = swap_accepts[max(len(swap_accepts) - 100, 0):]  # Select relevant data
                swap_acceptance_rate = np.mean(rel_accepts, axis=0)  # Average over multiple samples
                swap_acceptance_rate = np.mean(swap_acceptance_rate, axis=0)  # Average over multiple walkers
                swap_rate_diff = -np.diff(swap_acceptance_rate, axis=0)  # Compute the diff over the chains
                S = S + kappa * swap_rate_diff
                temp_diffs = self.temperatures
                temp_diffs[1:-1] = np.exp(S)
                self.temperatures = np.cumsum(temp_diffs)

        parameters = np.array(parameters)
        priors = np.array(priors)
        likelihoods = np.array(likelihoods)
        step_accepts = np.array(step_accepts)
        swap_accepts = np.array(swap_accepts)
        return parameters, priors, likelihoods, step_accepts, swap_accepts

    def step(self, params, prior, likelihood, index):
        move = np.random.normal(loc=0, scale=self.variance)
        proposal = params + move

        proposal_likelihood = self.log_likelihood(proposal)
        proposal_prior = self.log_prior(proposal)
        proposal_prob = self.beta * proposal_likelihood + proposal_prior

        prob = self.beta * likelihood + prior

        log_diff = proposal_prob - prob
        diff = np.exp(log_diff)
        u = np.random.uniform(size=(self.n_walkers, self.n_chains))
        accept = u < diff

        new_prior = np.where(accept, proposal_prior, prior)
        new_likelihood = np.where(accept, proposal_likelihood, likelihood)

        params_accepts = np.expand_dims(accept, -1)
        new_params = np.where(params_accepts, proposal, params)
        return new_params, new_prior, new_likelihood, accept

    def swap(self, params, prior, likelihood, index):
        log_diff = np.diff(likelihood, axis=-1)
        beta_diff = -np.diff(self.beta, axis=-1)

        log_criterion = beta_diff * log_diff
        criterion = np.exp(log_criterion)
        u = np.random.uniform(size=(self.n_walkers, self.n_chains - 1))
        # Ensure in the accepts step that a single chain does not swap to both adjacent chains (it should be possible to check this by using np.diff(accept) which should not yield 0 at a position including a 1 in accept
        proposed_accept = u < criterion
        self.swap_mask = np.roll(self.swap_mask, 1)
        swap_mask = self.swap_mask[:, :self.n_chains - 1]
        accept = np.logical_and(proposed_accept, swap_mask)
        # accept[:, i] defines whether to swap between chain i and i+1.

        # swap_matrice_1 is accept matrice with an additional all zeros entry
        swap_matrice_1 = np.concatenate((accept, np.zeros((self.n_walkers, 1))), axis=1)
        swap_matrice_2 = np.roll(swap_matrice_1, 1, axis=1)

        left_rolled_prior = np.roll(prior, -1, axis=1)
        right_rolled_prior = np.roll(prior, 1, axis=1)
        left_rolled_likelihood = np.roll(likelihood, -1, axis=1)
        right_rolled_likelihood = np.roll(likelihood, 1, axis=1)
        left_rolled_params = np.roll(params, -1, axis=1)
        right_rolled_params = np.roll(params, 1, axis=1)

        new_prior = np.where(swap_matrice_1, left_rolled_prior, prior)
        new_prior = np.where(swap_matrice_2, right_rolled_prior, new_prior)
        new_likelihood = np.where(swap_matrice_1, left_rolled_likelihood, likelihood)
        new_likelihood = np.where(swap_matrice_2, right_rolled_likelihood, new_likelihood)
        new_params = np.where(np.expand_dims(swap_matrice_1, -1), left_rolled_params, params)
        new_params = np.where(np.expand_dims(swap_matrice_2, -1), right_rolled_params, new_params)

        return new_params, new_prior, new_likelihood, accept


## Define Models

In [ ]:
def activatory_hill_equation(x, params):
    ymax, ymin, n, k = params
    y = ymin + (ymax - ymin) / (1 + np.power(k/x, n))
    return y

def inhibitory_hill_equation(x, params):
    ymax, ymin, n, k = params
    y = ymin + (ymax - ymin) / (1 + np.power(x/k, n))
    return y

## Define Loss Function Generator

In [ ]:
def get_loss_function(model, data):
    def loss_function(params):
        X, Y_true = data
        losses = []
        model_params = np.exp(params)        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            Y_pred = model(X, model_params)        
        diffs = np.log(Y_pred) - np.log(Y_true)         
        
        losses = np.power(diffs, 2)  
        nan_mask = np.logical_not(np.isnan(losses))
        loss = np.sum(losses[nan_mask])        
        
        return loss
        

    return loss_function

## Fit Inputs

In [ ]:
for construct_id in data_inputs:    
        
    cur_data = data_inputs[construct_id]
    inducer_concentration = cur_data["inducer concentrations"]
    output_RPU = cur_data["output"]


    data = inducer_concentration, list(map(lambda elem: np.median(elem), output_RPU))
    X, Y_true = data
    min_Y_true = np.nanmin(Y_true)
    max_Y_true = np.nanmax(Y_true)
    
    init_params = (min_Y_true, max_Y_true, 2, 1)
    

    loss_func = get_loss_function(model=activatory_hill_equation, data=data)
    

    def log_prior(params):
        # Define more precise prior for y_max and y_min with a broad distribution (y_max, y_max * 2). Check how proxies for n and k can be inferred directly from data

        a_min=np.log([max_Y_true, min_Y_true / 2, 10**(-20), 10**(-20)])
        a_max=np.log([max_Y_true * 2, min_Y_true, 10**10, 10**10])
        l_prior = - np.power(params - np.clip(params, a_min=a_min, a_max=a_max), 2) / 1
        l_prior = np.sum(l_prior, axis=-1)    
        
        return l_prior        
    
    def log_likelihood(params):    
        # Vectorization of loss function
        if len(params.shape) == 1:
            return -loss_func(params)
        else:
            log_likelihood_vals = np.empty(shape=params.shape[:2])
            for i1 in range(params.shape[0]):
               for i2 in range(params.shape[1]):
                   log_likelihood_vals[i1, i2] = -loss_func(params[i1, i2]) / 1
                
            return log_likelihood_vals
    
    
    
    pt = ParallelTempering(log_likelihood=log_likelihood, log_prior=log_prior, n_dim=len(init_params), 
                           n_walkers=n_walkers,
                           n_chains=n_chains)
    parameters, priors, likelihoods, step_accepts, swap_accepts = pt.run(initial_parameters=np.log(init_params), n_samples=n_samples,
                                                                             target_acceptance_ratio=0.4,
                                                                             adaptive_temperature=True)
    best_index = np.unravel_index(np.argmax(priors + likelihoods), likelihoods.shape)
    params = np.exp(parameters[*best_index])

    best_loss = loss_func(np.log(params))
    # ymax, ymin, n, k
    params_dict = {"y_max": params[0],
                   "y_min": params[1],
                   "n": params[2],
                   "k": params[3]}
    data_inputs[construct_id]["model information"] = {"model type": "ACTIVATORY HILL EQUATION",
                                                     "calibration loss": best_loss,
                                                     "parameters" : params_dict}
    print(f"Fitted parameters for input {construct_id} with loss {best_loss} ({params_dict})")
    
    
    # params = init_params

## Fit Outputs

In [ ]:
for construct_id in data_gates:
    cur_data = data_gates[construct_id]
    input_RPU = cur_data["input RPU"]
    output_RPU = cur_data["output"]   


    data = input_RPU, list(map(lambda elem: np.median(elem), output_RPU))
    X, Y_true = data
    min_Y_true = np.nanmin(Y_true)
    max_Y_true = np.nanmax(Y_true)
    
    init_params = (max_Y_true, min_Y_true, 2, 0.01)
    

    loss_func = get_loss_function(model=inhibitory_hill_equation, data=data)
    

    def log_prior(params):
        # Define more precise prior for y_max and y_min with a broad distribution (y_max, y_max * 2). Check how proxies for n and k can be inferred directly from data

        a_min=np.log([max_Y_true, min_Y_true / 2, 10**(-20), 10**(-20)])
        a_max=np.log([max_Y_true * 2, min_Y_true, 10**10, 10**10])
        l_prior = - np.power(params - np.clip(params, a_min=a_min, a_max=a_max), 2) / 1
        l_prior = np.sum(l_prior, axis=-1)    
        
        return l_prior        
    
    def log_likelihood(params):    
        # Vectorization of loss function
        if len(params.shape) == 1:
            return -loss_func(params)
        else:
            log_likelihood_vals = np.empty(shape=params.shape[:2])
            for i1 in range(params.shape[0]):
               for i2 in range(params.shape[1]):
                   log_likelihood_vals[i1, i2] = -loss_func(params[i1, i2]) / 1
                
            return log_likelihood_vals
    
    
    
    
    pt = ParallelTempering(log_likelihood=log_likelihood, log_prior=log_prior, n_dim=len(init_params), 
                           n_walkers=n_walkers,
                           n_chains=n_chains)
    parameters, priors, likelihoods, step_accepts, swap_accepts = pt.run(initial_parameters=np.log(init_params), n_samples=n_samples,
                                                                             target_acceptance_ratio=0.4,
                                                                             adaptive_temperature=True)
    best_index = np.unravel_index(np.argmax(priors + likelihoods), likelihoods.shape)
    params = np.exp(parameters[*best_index])

    best_loss = loss_func(np.log(params))
    # ymax, ymin, n, k
    params_dict = {"y_max": params[0],
                   "y_min": params[1],
                   "n": params[2],
                   "k": params[3]}
    data_gates[construct_id]["model information"] = {"model type": "INHIBITORY HILL EQUATION",
                                                     "calibration loss": best_loss,
                                                     "parameters" : params_dict}
    print(f"Fitted parameters for gate {construct_id} with loss {best_loss} ({params_dict})")
        

In [ ]:
# Code to visualize statistics of parallel tempering
if False:
    from numpy.lib.stride_tricks import sliding_window_view
    step_accepts_sliding_window = np.transpose(sliding_window_view(step_accepts[:, :, :], 100, axis=0),
                                               axes=(0, 3, 1, 2))
    step_accepts_avg = np.mean(step_accepts_sliding_window, axis=1)
    
    swap_accepts_sliding_window = np.transpose(sliding_window_view(swap_accepts, 10, axis=0), axes=(0, 3, 1, 2))
    swap_accepts_avg = np.mean(swap_accepts_sliding_window, axis=1)
    
    for iWalker in range(n_walkers):
        fig, axes = plt.subplots(ncols=2)
    
        for iChain in range(step_accepts_avg.shape[-1]):
            axes[0].plot(np.arange(2) * (len(step_accepts_avg) - 1),
                         np.ones(2) * 0.4 + (n_chains - iChain - 1), "k--", alpha=0.5)
            axes[0].plot(np.arange(len(step_accepts_avg)),
                         step_accepts_avg[:, iWalker, iChain] + (n_chains - iChain - 1), label=iChain, alpha=1)
    
        for iChain in range(swap_accepts_avg.shape[-1]):
            axes[1].plot(np.arange(2) * (len(swap_accepts_avg) - 1), np.ones(2) * 0 + (n_chains - iChain - 1), "r--",
                         alpha=0.5)
            axes[1].plot(np.arange(len(swap_accepts_avg)),
                         swap_accepts_avg[:, iWalker, iChain] + (n_chains - iChain - 1), label=iChain, alpha=1)
    
        ylim = axes[0].get_ylim()
        axes[1].set_ylim(ylim)
        axes[0].legend()
        axes[1].legend()
        plt.show()

# Visualize Data

## Visualize Constitutive Promoters

In [ ]:
key_median_pairs = [(key, np.median(data_constitutive[key]["output"])) for key in data_constitutive]
key_median_pairs = sorted(key_median_pairs, key=lambda elem: elem[1], reverse=False)
key_median_pairs = [elem[0] for elem in key_median_pairs]

input_device_keys = [key.split("-")[0] for key in key_median_pairs if "No" in key]
inducer_keys = [key.split("-")[1] for key in key_median_pairs]

input_device_keys = input_device_keys
inducer_keys = [elem for elem in inducer_order if elem in inducer_keys]

data = []
for iRow, input_device_key in enumerate(input_device_keys):
    cur_data = []
    for iCols, inducer_key in enumerate(inducer_keys):
        cur_data.append(data_constitutive[f"{input_device_key}-{inducer_key}"])
    data.append(cur_data)

data = np.array(data)

### Constitutive Promoters Comparison

In [ ]:
data_to_plot = list(map(lambda elem: elem["output"], data[:, 0]))

In [ ]:
figsize = (180, 90)
fig, ax = plt.subplots(figsize=mm_to_inch(figsize))

length = len(input_device_keys)
color = COLORS_MAIN[0]

for iR in range(length):
    input_device = input_device_keys[length - iR - 1]
    cur_data = data_to_plot[length - iR - 1][0]

    if plot_as_density and True:            
        kde = gaussian_kde(cur_data, bw_method=kde_bandwith)
        kde_vals = kde(histogram_bins)
        kde_vals = kde_vals / np.max(kde_vals) * 0.75
        ax.plot(histogram_bins, kde_vals + iR, color=color)
        ax.fill_between(histogram_bins, np.ones(shape=histogram_bins.shape) * iR, kde_vals + iR, color=color, alpha=0.5)
    else:
        hist, bins = np.histogram(cur_data, bins=histogram_bins)
        hist = hist  / np.max(hist) * 1.5
        ax.bar(bins[:-1], hist, np.diff(histogram_bins), color=color, alpha=0.5, align="edge", bottom=iR)    
        # ax.plot(bins, np.ones(len(bins)) * iR, color=color) 
        ax.plot(np.repeat(bins, 2)[1:-1], np.repeat(hist, 2) + iR, linewidth=0.6, color=color, solid_capstyle='butt', solid_joinstyle="miter") 
        
    
    
    ax.text(bins[0], iR + 0.1, f"{input_device} ({np.median(cur_data):.2f})", va='bottom', ha='left')#, fontsize=10)

    ax.set_xlabel("[RPU]")
    ax.set_yticks([])
    

# ax.plot(np.flip(list(map(np.median, data_to_plot))), range(length), color=COLORS_MAIN[0])
    

ax.set_xscale("log")
plt.savefig(f"{figure_dir}/constitutive_promoters_histograms{figure_extension}")
plt.show()


### Constitutive Promoters Cross Reactivity

In [ ]:
data_to_plot = np.zeros(shape=data.shape)
for iRow in range(data.shape[0]):
    for iCol in range(data.shape[1]):
        data_to_plot[iRow, iCol] = np.median(data[iRow, iCol]["output"])

data_to_plot = data_to_plot.transpose()

fig_size = np.array([180, 90])
fig_size = mm_to_inch(fig_size)
fig, ax = plt.subplots(figsize=fig_size)
im = ax.imshow(data_to_plot, cmap=COLORMAP_BLUE, interpolation="none", origin="upper", rasterized=True, norm=matplotlib.colors.SymLogNorm(vmax=10**1, linthresh=0.001))


x_ticks = np.arange(data_to_plot.shape[1])
y_ticks = np.arange(data_to_plot.shape[0])
ax.set_xticks(x_ticks)
ax.set_yticks(y_ticks)
ax.set_xticklabels(input_device_keys,rotation=45)
ax.set_yticklabels(inducer_keys)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)

ax.set_xticks(np.arange(data_to_plot.shape[1] + 1) - 0.5, minor=True)
ax.set_yticks(np.arange(data_to_plot.shape[0] + 1) - 0.5, minor=True)
ax.grid(which="minor", color="#808080", linestyle='-', linewidth=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size=0.2, pad=0.1)
cbar = plt.colorbar(im, cax=cax)
cbar.set_label("Output [RPU]")
cbar.set_ticks([10**(-1), 10**0, 10**1])


plt.tight_layout()
plt.savefig(f"{figure_dir}/constitutive_promoter_cross_reactivity{figure_extension}")
plt.show()

### Save data to table

In [ ]:
table_rep = []

header = ["Device"] + inducer_keys[:1]
table_rep.append(header)

length = len(input_device_keys)
for iR in range(length):
    input_device = input_device_keys[iR]
    row_vals = [input_device]
    # for iC in range(data.shape[1]):
        # cur_data = data[iR][iC]
    cur_data = data[iR][0]
    cur_val = np.median(cur_data["output"][0])
    row_vals.append(f"{cur_val:.4f}")
    table_rep.append(row_vals)
    
    
table_str = "\n".join([", ".join(row) for row in table_rep])
print(table_rep)
with open("figures/constitutive_promoters_cross_reactivity.csv", "w") as file:
    file.write(table_str)

In [ ]:
# Visualize consistency of reference promoter
ref_data = data_reference["pJCM434"]
const_data = data_constitutive["J23101-No"]
const_data_AU = const_data["data"]

figsize = (180, 60)

inducer_concentration = ref_data["inducer concentrations"]
output_AU = ref_data["data"]

fig, axes = plt.subplots(ncols=len(inducer_concentration), sharey=True, figsize=mm_to_inch(figsize))
fig.subplots_adjust(wspace=0)

is_first = True
iL = 0

masks = np.array(cur_data["replicates merge info"])
for ax, ind_level, out_levels in zip(axes, inducer_concentration, output_AU):    


    ax.hist(const_data_AU, 
        orientation="horizontal", 
        color=COLORS_MAIN[1], 
        alpha=0.4,
        bins=np.logspace(-1, 5, 200),#histogram_bins, 
        density=hist_as_density,
        histtype="stepfilled",
        ec="k")
    
    ax.hist(out_levels, 
        orientation="horizontal", 
        color=COLORS_MAIN[0], 
        alpha=0.4,
        bins=np.logspace(-1, 5, 200),#histogram_bins, 
        density=hist_as_density,
        histtype="stepfilled",
        ec="k")



    # print(ind_level, np.mean(cur_data[ind_level]))

    if False:
        for iR, replicate_id in enumerate(cur_data["replicates"]):
            replicate_output_AU = cur_data["replicates"][replicate_id]["output"]        
            color = "black" if masks[iL, iR] else COLORS[2]
            ax.hist(replicate_output_AU[iL], 
                    orientation="horizontal", 
                    color=color, 
                    bins=histogram_bins, 
                    density=hist_as_density, 
                    alpha=0.3,
                   histtype="stepfilled",
                   ec="k")    

    xlim = ax.get_xlim()
    ax.set_xlim((xlim[0], xlim[1] * 1.2))
    ax.set_yscale("log")
    # ax.set_title(inducer_concentration)
    ax.set_xticks([])  

    if is_first:
       ax.set_ylabel("Fluorescence [AU]")
       ax.tick_params(axis='y', which='both', direction="in", left=True)            
       is_first = False
    else:
       ax.tick_params(axis='y', which='both', left=False)    
        
    iL += 1
    ax.set_xlabel(f"{ind_level}")# {cur_data['inducer unit']}")
    
plt.suptitle(construct_id)
# plt.savefig(figure_dir + "reference_" + construct_id + "_histograms" + figure_extension)
plt.show()



# Visualize Cross Reactivity

In [ ]:
cross_reactivity_keys = list(data_inputs_cross_reactivity.keys())
input_device_keys = set()
inducer_keys = set()
for key in cross_reactivity_keys:
    input_device_key, inducer_key = key.split("-")
    input_device_keys.add(input_device_key)
    inducer_keys.add(inducer_key)

input_device_keys = [elem for elem in input_sensor_order if elem in input_device_keys]
inducer_keys = [elem for elem in inducer_order if elem in inducer_keys]

data = []
for iRow, input_device_key in enumerate(input_device_keys):
    cur_data = []
    for iCols, inducer_key in enumerate(inducer_keys):
        cur_data.append(data_inputs_cross_reactivity[f"{input_device_key}-{inducer_key}"])
    data.append(cur_data)

data = np.array(data)

In [ ]:
data_to_plot = np.zeros(shape=data.shape)
for iRow in range(data.shape[0]):
    for iCol in range(data.shape[1]):
        data_to_plot[iRow, iCol] = np.median(data[iRow, iCol]["output"])

data_to_plot = data_to_plot.transpose()

fig_size = (60, 42)
fig_size = mm_to_inch(fig_size)
fig, ax = plt.subplots(figsize=fig_size)
im = ax.imshow(data_to_plot, cmap=COLORMAP_BLUE, interpolation="none", origin="upper", rasterized=True, norm=matplotlib.colors.SymLogNorm(linthresh=0.001))


x_ticks = np.arange(data_to_plot.shape[1])
y_ticks = np.arange(data_to_plot.shape[0])
ax.set_xticks(x_ticks)
ax.set_yticks(y_ticks)
ax.set_xticklabels(input_device_keys)
ax.set_yticklabels(inducer_keys)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)

ax.set_xticks(np.arange(data_to_plot.shape[1] + 1) - 0.5, minor=True)
ax.set_yticks(np.arange(data_to_plot.shape[0] + 1) - 0.5, minor=True)
ax.grid(which="minor", color="#808080", linestyle='-', linewidth=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size=0.1, pad=0.1)
cbar = plt.colorbar(im, cax=cax)
cbar.set_label("Output [RPU]", fontsize=6)
cbar.set_ticks([10**(-2), 10**(-1), 10**0, 10**1])
# cbar.set_scale("log")

plt.tight_layout()
plt.savefig(f"{figure_dir}/cross_reactivity{figure_extension}")
plt.show()

## Visualize Inputs

In [ ]:
figsize = (50, 40)
linthreshs = [1, 1, 0.1]
for construct_id in data_inputs:
    cur_data = data_inputs[construct_id]
    inducer_concentration = cur_data["inducer concentrations"]
    output_RPU = cur_data["output"]
    
    fig, ax = plt.subplots(figsize=mm_to_inch(figsize))


    if "model information" in cur_data:
        params_dict = cur_data["model information"]["parameters"]
        X = histogram_bins
        params = [params_dict[param_id] for param_id in ["y_max", "y_min", "n", "k"]]
        Y_model = activatory_hill_equation(X, params)
        ax.plot(X, Y_model, color="#808080", label="Model", zorder=-5)
        

    
    ax.scatter(inducer_concentration, list(map(np.median, output_RPU)), color=COLORS_MAIN[0], s=10) 
    masks = np.array(cur_data["replicates merge info"], dtype=bool)
    for iR, replicate_id in enumerate(cur_data["replicates"]):
        replicate_output_RPU = cur_data["replicates"][replicate_id]["output"]                
        Y = list(map(np.median, [elem for flag, elem in zip(masks[:, iR], replicate_output_RPU) if flag]))        
        ax.scatter(np.array(inducer_concentration)[masks[:, iR]], Y, color="k", marker=".", s=3) 
        Y = list(map(np.median, [elem for flag, elem in zip(masks[:, iR], replicate_output_RPU) if not flag]))
        ax.scatter(np.array(inducer_concentration)[np.logical_not(masks[:, iR])], Y, color=COLOR_GRAY, marker=".", s=3) 
    # ax.plot(inducer_concentration,  list(map(np.median, output_RPU)), color=COLORS_MAIN[0])


    linthresh = 10**np.floor(np.log10(inducer_concentration[1] / 4))
    
    ax.set_xlim((-linthresh/2, np.max(inducer_concentration) * 2))
    ax.set_ylim((lim[0], 10**1.4))
    ax.set_xlabel(f"{cur_data['inducer molecule']} [{cur_data['inducer unit']}]")
    ax.set_ylabel("Output [RPU]")    
    ax.set_xscale("symlog", linthresh=linthresh)
    ax.set_yscale("log")
    ax.tick_params(axis='both', which='both', direction="in", bottom=True, left=True)
    # ax.set_title(construct_id)

    plt.savefig(figure_dir + "input_" + construct_id + "_response_curve" + figure_extension)

In [ ]:
figsize = (180, 60)

cur_colors = ["#EB99B4", "#F5B199", "#FFCC99"]
for construct_id in data_inputs:
    cur_data = data_inputs[construct_id]
    inducer_concentrations = cur_data["inducer concentrations"]
    output_RPU = cur_data["output"]

    fig, axes = plt.subplots(ncols=len(inducer_concentrations), sharey=True, figsize=mm_to_inch(figsize))
    fig.subplots_adjust(wspace=0)

    is_first = True
    iL = 0
    
    masks = np.array(cur_data["replicates merge info"])
    for ax, ind_level, out_levels in zip(axes, inducer_concentrations, output_RPU):    
            
        ax.hist(out_levels, 
                orientation="horizontal", 
                color=COLORS_MAIN[0], 
                bins=histogram_bins, 
                density=hist_as_density,
                histtype="stepfilled",
                linewidth=0.4,
                ec="k")

        for iR, replicate_id in enumerate(cur_data["replicates"]):            
            replicate_output_RPU = cur_data["replicates"][replicate_id]["output"]        
            color = COLOR_REPLICATES[iR]
            alpha = 0.4 if masks[iL, iR] else 0.2
                
            ax.hist(replicate_output_RPU[iL], 
                    orientation="horizontal", 
                    color=color, 
                    bins=histogram_bins, 
                    density=hist_as_density, 
                    alpha=alpha,
                    histtype="stepfilled",
                    linewidth=0.2,
                   )
            if True and not masks[iL, iR]:
                ax.hist(replicate_output_RPU[iL], 
                    orientation="horizontal", 
                    color=COLOR_GRAY,#color, 
                    bins=histogram_bins, 
                    density=hist_as_density, 
                    alpha=0.2,
                    histtype="stepfilled",
                    linewidth=0.2,
                    )    

        xlim = ax.get_xlim()
        ax.set_xlim((xlim[0], xlim[1] * 1.2))
        ax.set_yscale("log")
        # ax.set_title(inducer_concentrations)
        ax.set_xticks([])  

        if is_first:
           ax.set_ylabel("Output [RPU]")
           ax.tick_params(axis='y', which='both', direction="in", left=True)            
           is_first = False
        else:
           ax.tick_params(axis='y', which='both', left=False)    
            
        iL += 1
        ax.set_xlabel(f"{ind_level}")
        
    plt.suptitle(construct_id)
    plt.savefig(figure_dir + "input_" + construct_id + "_histograms" + figure_extension)
    plt.show()

## Visualize Gates

### Visualization of Individual Gates

In [ ]:
from matplotlib.ticker import AutoMinorLocator

figsize = (48, 50)
figsize = (88, 86)
# figsize = (120, 120)
for construct_id in data_gates:
    cur_data = data_gates[construct_id]
    input_RPU = cur_data["input RPU"]
    output_RPU = cur_data["output"]
    
    
    
    fig, ax = plt.subplots(figsize=mm_to_inch(figsize))
    ax.scatter(input_RPU, list(map(np.median, output_RPU)), color=COLORS_MAIN[0], s=10) 
    # ax.plot(input_RPU,  list(map(np.median, output_RPU)), color=COLORS_MAIN[0])

    if "model information" in cur_data:
        params_dict = cur_data["model information"]["parameters"]
        X = np.logspace(-5, 4, 100)#histogram_bins
        params = [params_dict[param_id] for param_id in ["y_max", "y_min", "n", "k"]]
        Y_model = inhibitory_hill_equation(X, params)
        ax.plot(X, Y_model, color=COLORS_MAIN[1], label="Model")
        

    masks = np.array(cur_data["replicates merge info"], dtype=bool)
    for iR, replicate_id in enumerate(cur_data["replicates"]):
        replicate_output_RPU = cur_data["replicates"][replicate_id]["output"]        
        Y = list(map(np.median, [elem for flag, elem in zip(masks[:, iR], replicate_output_RPU) if flag]))
        ax.scatter(np.array(input_RPU)[masks[:, iR]], Y, color="k", marker=".", s=3) 
        Y = list(map(np.median, [elem for flag, elem in zip(masks[:, iR], replicate_output_RPU) if not flag]))
        ax.scatter(np.array(input_RPU)[np.logical_not(masks[:, iR])], Y, color=COLORS[2], marker=".", s=3)  

    ax.set_xticks([10**(-3), 10**(-1), 10**1])
    ax.set_yticks([10**(-3), 10**(-1), 10**1])    
    ax.set_xlim(lim)
    ax.set_ylim(lim)
    ax.set_xlabel("Input [RPU]")
    ax.set_ylabel("Output [RPU]")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.tick_params(axis='both', which='both', direction="in", bottom=True, left=True)
    ax.set_box_aspect(1)
    ax.set_title(construct_id)
    plt.savefig(f"{figure_dir}gate_{construct_id}_response_curve_{"-".join(list(map(str,figsize)))}{figure_extension}")
    plt.show()

    

In [ ]:
figsize = (48, 50)
# figsize = (88, 86)
# figsize = (100, 50)

for construct_id in data_gates:
    cur_data = data_gates[construct_id]
    inducer_concentrations = cur_data["inducer concentrations"]
    input_RPU = cur_data["input RPU"]
    output_RPU = cur_data["output"]

    fig, axes = plt.subplots(ncols=len(input_RPU), sharey=True, figsize=mm_to_inch(figsize))
    fig.subplots_adjust(wspace=0)


    is_first = True
    iL = 0
    
    masks = np.array(cur_data["replicates merge info"], dtype=bool)
    for ax, ind_level, input_level, out_levels in zip(axes, inducer_concentrations, input_RPU, output_RPU):    
            
        ax.hist(out_levels, 
           orientation="horizontal", 
           color=COLORS_MAIN[0], 
           bins=histogram_bins, 
           density=hist_as_density,
               histtype="stepfilled",
                linewidth=0.4,
               ec="k")

        for iR, replicate_id in enumerate(cur_data["replicates"]):
            replicate_output_RPU = cur_data["replicates"][replicate_id]["output"]        
            color = COLOR_REPLICATES[iR]
            alpha = 0.4 if masks[iL, iR] else 0.2
                
            ax.hist(replicate_output_RPU[iL], 
                    orientation="horizontal", 
                    color=color, 
                    bins=histogram_bins, 
                    density=hist_as_density, 
                    alpha=alpha,
                    histtype="stepfilled",
                    linewidth=0.2,
                   )
            if True and not masks[iL, iR]:
                ax.hist(replicate_output_RPU[iL], 
                    orientation="horizontal", 
                    color=COLOR_GRAY,#color, 
                    bins=histogram_bins, 
                    density=hist_as_density, 
                    alpha=0.2,
                    histtype="stepfilled",
                    linewidth=0.2,
                    )    

        xlim = ax.get_xlim()
        ax.set_xlim((xlim[0], xlim[1] * 1.2))
        ax.set_yscale("log")
        # ax.set_xlabel(f"{int(ind_level)}", fontsize=6)#:.3f}")
        ax.set_xticks([])  

        if is_first:
            ax.set_ylabel("Output [RPU]")
            ax.tick_params(axis='y', which='both', direction="in", left=True)            
            is_first = False
        else:
            ax.tick_params(axis='y', which='both', left=False)            
        iL += 1
            
    plt.suptitle(construct_id)
    plt.savefig(f"{figure_dir}gate_{construct_id}_histograms_{"-".join(list(map(str,figsize)))}{figure_extension}")
    plt.show()

### Visualization of Gate Library

In [ ]:
gate_id_median_pairs = [(gate_id, list(map(np.median, data_gates[gate_id]["output"]))) for gate_id in data_gates]
gate_dynamic_ranges = [(elem[0], np.nanmax(elem[1])/np.nanmin(elem[1])) for elem in gate_id_median_pairs]
gate_dynamic_ranges = sorted(gate_dynamic_ranges, key=lambda elem: elem[1], reverse=True)
gate_dynamic_ranges

In [ ]:


figsize = (88, 86)

fig, ax = plt.subplots(figsize=mm_to_inch(figsize))
for iX, gate_tup in enumerate(gate_dynamic_ranges):
    if iX >= present_top_k:
        break
    construct_id, dr = gate_tup
    cur_data = data_gates[construct_id]
    input_RPU = cur_data["input RPU"]
    output_RPU = cur_data["output"]

#    if "model information" in cur_data:
    params_dict = cur_data["model information"]["parameters"]
    X = np.logspace(-5, 4, 100)#histogram_bins
    params = [params_dict[param_id] for param_id in ["y_max", "y_min", "n", "k"]]
    Y_model = inhibitory_hill_equation(X, params)
    ax.plot(X, Y_model, color=COLORS[iX], label=f"{construct_id}")

    
    # ax.scatter(input_RPU, list(map(np.median, output_RPU)), color=COLORS[iX], label=construct_id, s=10) 
    # ax.plot(input_RPU,  
    #         list(map(np.median, output_RPU)), 
    #         ["--", "-.", "--."][iX % 3], 
    #         color=(COLORS + COLORS_DARK)[(iX) % (len(COLORS) + len(COLORS_DARK) - 1)],
    #         label=construct_id)

ax.set_xlabel("Input [RPU]")
ax.set_ylabel("Output [RPU]")
ax.set_xscale("log")
ax.set_yscale("log")            

ax.tick_params(axis='x', which='both', direction="in", left=True)   
ax.tick_params(axis='y', which='both', direction="in", left=True)            
ax.set_box_aspect(1)         

ax.set_xlim(lim)
ax.set_ylim(lim)
        
ax.legend()
plt.savefig(figure_dir +  "gates_all_response_curves" + figure_extension)

### Gate Library Parameters

In [ ]:
table_rep = []

params_order = ["y_max", "y_min", "n", "k"]
header = ["Plasmid", "Gate"] + params_order + ["Dynamic Range"]
table_rep.append(header)

for construct_id in data_inputs:
    cur_data = data_inputs[construct_id]
    
    params_dict = cur_data["model information"]["parameters"]
    params = [params_dict[param_id] for param_id in params_order]        
    
    
    row_vals = [construct_id, ""]
    row_vals += [f"{val:.3f}" for val in params]
    row_vals += [f"{params[0] / params[1] :.3f}"]
    
    table_rep.append(row_vals)
    
    
table_str = "\n".join([", ".join(row) for row in table_rep])
print(table_rep)
with open(f"{figure_dir}/input_hill_parameters.csv", "w") as file:
    file.write(table_str)

In [ ]:
table_rep = []

params_order = ["y_max", "y_min", "n", "k"]
header = ["Plasmid", "Gate"] + params_order + ["Dynamic Range"]
table_rep.append(header)

for construct_id in data_gates:
    cur_data = data_gates[construct_id]
    
    params_dict = cur_data["model information"]["parameters"]
    params = [params_dict[param_id] for param_id in params_order]        
    
    
    row_vals = [construct_id, ""]
    row_vals += [f"{val:.3f}" for val in params]
    row_vals += [f"{params[0] / params[1] :.3f}"]
    
    table_rep.append(row_vals)
    
    
table_str = "\n".join([", ".join(row) for row in table_rep])
print(table_rep)
with open(f"{figure_dir}/gate_parameters.csv", "w") as file:
    file.write(table_str)